Tutorial came from https://medium.com/analytics-vidhya/sentiment-analysis-on-amazon-reviews-using-tf-idf-approach-c5ab4c36e7a1

credits goes to him

In [127]:
import urllib.request
import gzip
import shutil
import os
from IPython.display import Markdown as md

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\Yu
[nltk_data]     Jin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Download the dataset
This dataset contains reviews found on amazon with ratings, we will use the reviews to train a model to classify the sentiment of a review posted as positive or negative.

In [8]:
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz'
filename = 'amazon_reviews_us_Jewelry_v1_00.tsv.gz'
if not os.path.exists(filename):
    urllib.request.urlretrieve(url, filename)
    print("Successfully Downloaded {0}".format(filename))

    if filename.endswith(".gz"):
        with gzip.open(filename, 'rb') as f_in:
            with open("data.tsv", 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    os.remove(filename)

In [25]:
df=pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False)

D:\WPy64-38100\python-3.8.10.amd64\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 10437: expected 15 fields, saw 22\nSkipping line 10443: expected 15 fields, saw 22\nSkipping line 19872: expected 15 fields, saw 22\nSkipping line 20055: expected 15 fields, saw 22\nSkipping line 20107: expected 15 fields, saw 22\nSkipping line 20167: expected 15 fields, saw 22\nSkipping line 53858: expected 15 fields, saw 22\n'
b'Skipping line 72173: expected 15 fields, saw 22\nSkipping line 84308: expected 15 fields, saw 22\nSkipping line 92156: expected 15 fields, saw 22\nSkipping line 97791: expected 15 fields, saw 22\nSkipping line 106812: expected 15 fields, saw 22\n'
b'Skipping line 138899: expected 15 fields, saw 22\nSkipping line 145840: expected 15 fields, saw 22\nSkipping line 192385: expected 15 fields, saw 22\n

In [26]:
df.head(3)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0.0,0.0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0.0,0.0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0.0,0.0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31


# Text pre-processing

We require only the review_body, star_rating columns which describes the reviews, star rating of each review respectively.

In [29]:
df=df[["review_body", "star_rating"]]
df

,review_body,star_rating
0,so beautiful even tho clearly not high end ......,5
1,"Great product.. I got this set for my mother, ...",5
2,Exactly as pictured and my daughter's friend l...,5
3,Love it. Fits great. Super comfortable and nea...,5
4,Got this as a Mother's Day gift for my Mom and...,5
...,...,...
1766987,It is nice looking and everything (it is sterl...,4
1766988,"my boyfriend bought me this last christmas, an...",4
1766989,This is a great way to quickly start learning ...,4
1766990,the 14kt gold earrings look remarkable...would...,5


Remove the Null, missing values and reset the index

In [30]:
df = df.dropna()
df = df.reset_index(drop=True)
df

,review_body,star_rating
0,so beautiful even tho clearly not high end ......,5
1,"Great product.. I got this set for my mother, ...",5
2,Exactly as pictured and my daughter's friend l...,5
3,Love it. Fits great. Super comfortable and nea...,5
4,Got this as a Mother's Day gift for my Mom and...,5
...,...,...
1766743,It is nice looking and everything (it is sterl...,4
1766744,"my boyfriend bought me this last christmas, an...",4
1766745,This is a great way to quickly start learning ...,4
1766746,the 14kt gold earrings look remarkable...would...,5


Now we have 17,66,748 reviews. The reviews with star rating 4,5 are labelled as positive reviews and 1,2 are labelled as negative reviews. Remove the reviews with star rating 3 as they are considered as neutral.

In [32]:
df['star_rating']=df['star_rating'].astype(int) #convert the star_rating column to int
df=df[df['star_rating']!=3]
df['label']=np.where(df['star_rating']>=4,1,0) #1-Positve,0-Negative
df

C:\Users\YUJIN~1\AppData\Local\Temp/ipykernel_14892/1618018613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating']=df['star_rating'].astype(int) #convert the star_rating column to int


,review_body,star_rating,label
0,so beautiful even tho clearly not high end ......,5,1
1,"Great product.. I got this set for my mother, ...",5,1
2,Exactly as pictured and my daughter's friend l...,5,1
3,Love it. Fits great. Super comfortable and nea...,5,1
4,Got this as a Mother's Day gift for my Mom and...,5,1
...,...,...,...
1766743,It is nice looking and everything (it is sterl...,4,1
1766744,"my boyfriend bought me this last christmas, an...",4,1
1766745,This is a great way to quickly start learning ...,4,1
1766746,the 14kt gold earrings look remarkable...would...,5,1


Number of reviews group by star rating

In [35]:
df['star_rating'].value_counts()

5    1080871
4     270424
1     155002
2     100797
Name: star_rating, dtype: int64

Now we are creating the model by considering the 100000 reviews. In the 100,000 reviews 50,000 are positive and 50,000 are negative.
I am shuffling the review as to take random 100,000 reviews from 1,607,094. You can ignore if you don’t want to shuffle.

In [45]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle
data=df[df['label']==0][:50000]
data=data.append(df[df['label']==1][:50000])
data = data.reset_index(drop=True)
display(data['label'].value_counts())
data

0    50000
1    50000
Name: label, dtype: int64

,review_body,star_rating,label
0,Pretty but too flimsy. Not what I expected,1,0
1,"Bad quality, it started to fade after use...",1,0
2,Very small. Much smaller the looked on tv. Wil...,2,0
3,fake not silver,1,0
4,Cheap looking! What did I expect for it's price?,2,0
...,...,...,...
99995,I ordered a silver bird charm to give to my da...,5,1
99996,"Very nice necklace, my fiancé loved it. But th...",4,1
99997,Necklace appears as expected. Have not given t...,5,1
99998,This was a gift for my sister from my nephew f...,5,1


The first step is convert the all reviews into the lower case.

In [46]:
data["pre_process"] = data["review_body"].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

Remove the HTML tags and URLs from the reviews.

In [47]:
data["pre_process"]=data["pre_process"].apply(lambda x: BeautifulSoup(x).get_text())
data["pre_process"]=data["pre_process"].apply(lambda x: re.sub(r"http\S+", "", x))

C:\Users\Yu Jin\AppData\Roaming\Python\Python38\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "https://www.amazon.com/review/review-your-purchases/ref=oh_aui_rev_shipment_o00_s00?_encoding=utf8&asins=b01444jlxe&channel=yacc-wr#" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\Users\Yu Jin\AppData\Roaming\Python\Python38\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "http://www.amazon.com/review/creata-review?ie=utf8&asin=b010xgqugs" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


Perform the Contractions on the reviews.  
Example: it won’t be converted as it will not be

In [48]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"would't", "would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

Remove non-alpha characters

In [49]:
data

,review_body,star_rating,label,pre_process
0,Pretty but too flimsy. Not what I expected,1,0,pretty but too flimsy. not what i expected
1,"Bad quality, it started to fade after use...",1,0,"bad quality, it started to fade after use..."
2,Very small. Much smaller the looked on tv. Wil...,2,0,very small. much smaller the looked on tv. wil...
3,fake not silver,1,0,fake not silver
4,Cheap looking! What did I expect for it's price?,2,0,cheap looking! what did i expect for it is price?
...,...,...,...,...
99995,I ordered a silver bird charm to give to my da...,5,1,i ordered a silver bird charm to give to my da...
99996,"Very nice necklace, my fiancé loved it. But th...",4,1,"very nice necklace, my fiancé loved it. but th..."
99997,Necklace appears as expected. Have not given t...,5,1,necklace appears as expected. have not given t...
99998,This was a gift for my sister from my nephew f...,5,1,this was a gift for my sister from my nephew f...


Remove whitespaces and punctuations at the start of the string which hinders wordtokenizer. Then remove non-alpha characters.

In [93]:
data['pre_process']=data['pre_process'].apply(lambda x: x.lstrip(string.punctuation + " "))

In [94]:
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

Remove extra spaces between the words

In [95]:
data["pre_process"]=data["pre_process"].apply(lambda x: re.sub(" +", " ", x))

Remove the stop words by using the NLTK package

In [97]:

stop = stopwords.words('english')
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

Perform lemmatization using the wordnet lemmatizer

In [101]:

lemmatizer = WordNetLemmatizer()
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

The final Pre-processed reviews look as:
Original: It looks better in picture. In reality, printing quality is not so good, and I don’t feel any coating.
Preprocessed: look better picture reality printing quality good feel coating.

In [102]:
data

,review_body,star_rating,label,pre_process
0,Pretty but too flimsy. Not what I expected,1,0,pretty flimsy expected
1,"Bad quality, it started to fade after use...",1,0,bad quality started fade use
2,Very small. Much smaller the looked on tv. Wil...,2,0,small much smaller looked tv send backpretty s...
3,fake not silver,1,0,fake silver
4,Cheap looking! What did I expect for it's price?,2,0,cheap looking expect price
...,...,...,...,...
99995,I ordered a silver bird charm to give to my da...,5,1,ordered silver bird charm give daughter mark s...
99996,"Very nice necklace, my fiancé loved it. But th...",4,1,nice necklace fianc loved chain really short f...
99997,Necklace appears as expected. Have not given t...,5,1,necklace appears expected given gift yet know ...
99998,This was a gift for my sister from my nephew f...,5,1,gift sister nephew mother day loved well made ...


# Feature Extraction  

TF-IDF: It is a method of extracting the features from the text data. TF stands for Term Frequency and IDF stands for Inverse Document Frequency.  

Term Frequency: Number of times word occurs in a review. For an example consider 2 reviews where w1,w2.. represents the words in both reviews and table defines the frequency of words in the particular review.

![title](images/TD-IDF1.png)

IDF is computed as

idf(t)  
= log [ n / df(t) ] + 1  
= log[ number of documents / number of documents containing the term]+1  
    
![title](images/TD-IDF2.png)

If smooth_idf=True.  
Smooth-IDF = log [ n / df(t) +1 ] + 1

So TF-IDF is calculated by multiplying the term frequency by the inverse document frequency (occurence * descriptiveness of word)
![title](images/TD-IDF3.png)


In [105]:

X_train, X_test, Y_train, Y_test = train_test_split(data['pre_process'], data['label'], test_size=0.25, random_state=30)
print("Train: ", X_train.shape, Y_train.shape,"Test: ", (X_test.shape, Y_test.shape))

Train:  (75000,) (75000,) Test:  ((25000,), (25000,))


Convert the pre-processed reviews into TF-IDF vectors

In [108]:
print("TFIDF Vectorizer……")
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


We can see that the vectorizer forms a collection of sparse vectors. nonzero() shows the indexes where the vector containa a value.

In [117]:
print(tf_x_train[0].toarray())
print(tf_x_train[0].toarray().nonzero())

[[0. 0. 0. ... 0. 0. 0.]]
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64), array([ 6229,  6239,  7447,  7911, 10608, 13774, 14183, 14627, 15781,
       16017, 18889, 21413, 22324, 22407, 22466, 22748, 24324, 31983],
      dtype=int64))


## SVM
Implementing SVM with sklearn for classification

In [141]:

clf = LinearSVC(random_state=0)
clf.fit(tf_x_train,Y_train)

LinearSVC(random_state=0)

Predicting on the Test data

In [142]:
y_test_pred=clf.predict(tf_x_test)

In [143]:

report=classification_report(Y_test, y_test_pred,output_dict=True)

In [144]:
report

{'0': {'precision': 0.9098573792451343,
  'recall': 0.9181774809160306,
  'f1-score': 0.9139984960620572,
  'support': 12576},
 '1': {'precision': 0.9164026322203266,
  'recall': 0.9079201545396007,
  'f1-score': 0.9121416730683701,
  'support': 12424},
 'accuracy': 0.91308,
 'macro avg': {'precision': 0.9131300057327305,
  'recall': 0.9130488177278157,
  'f1-score': 0.9130700845652137,
  'support': 25000},
 'weighted avg': {'precision': 0.913110108163686,
  'recall': 0.91308,
  'f1-score': 0.9130757293071143,
  'support': 25000}}

In [145]:
SVM_accuracy = report["accuracy"]
md("By Using the SVM classifier we got an accuracy of {}%".format(round(SVM_accuracy*100, 2)))

By Using the SVM classifier we got an accuracy of 91.31%

# Logistic Regression
Logistic regression is implemented using sklearn

In [146]:

clf = LogisticRegression(max_iter=1000,solver='saga')
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=1000, solver='saga')

Predicting the test data

In [147]:
y_test_pred=clf.predict(tf_x_test)
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [148]:
report

{'0': {'precision': 0.9079749804534792,
  'recall': 0.923425572519084,
  'f1-score': 0.9156351021051802,
  'support': 12576},
 '1': {'precision': 0.9211302211302211,
  'recall': 0.90526400515132,
  'f1-score': 0.9131281968011692,
  'support': 12424},
 'accuracy': 0.9144,
 'macro avg': {'precision': 0.9145526007918502,
  'recall': 0.914344788835202,
  'f1-score': 0.9143816494531747,
  'support': 25000},
 'weighted avg': {'precision': 0.914512608860193,
  'recall': 0.9144,
  'f1-score': 0.9143892704452989,
  'support': 25000}}

In [149]:
Logistic_accuracy = report["accuracy"]
md("By Using the Logistic regression classifier we got an accuracy of {}%".format(round(Logistic_accuracy*100, 2)))

By Using the Logistic regression classifier we got an accuracy of 91.44%